In [83]:
import re
import time
import pandas as pd
import numpy as np
import gensim.downloader as api

from nltk.tokenize import WordPunctTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [84]:
# Read data

training_data = pd.read_parquet('./data/train.parquet', engine='pyarrow')
test_data = pd.read_parquet('./data/test.parquet', engine='pyarrow')

##### EDA

In [85]:
# Common info
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48000 entries, 34862324 to 40415937
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    48000 non-null  object
 1   Tags    48000 non-null  object
 2   Title   48000 non-null  object
 3   target  48000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [86]:
# Classes are balanced:
training_data.target.value_counts()

2    16000
1    16000
0    16000
Name: target, dtype: int64

In [87]:
# There are no missing values in the dataset:
training_data.isna().sum()

Body      0
Tags      0
Title     0
target    0
dtype: int64

##### Сlassic vectorization

In [88]:
# Data preprocessing

text = []
target = []
for index, row in training_data.iterrows():
    tmp = row['Body'] + row['Title'] + row['Tags']
    text.append(tmp)
    target.append(row['target'])

test_text = []
for index, row in test_data.iterrows():
    tmp = row['Body'] + row['Title'] + row['Tags']
    test_text.append(tmp)
    
inputs = {}

In [91]:
# BOW

# create the transform
vectorizer = CountVectorizer(dtype=np.uint16)
# tokenize and build vocab
vectorizer.fit(text)
# encode
bow_vector = vectorizer.transform(text)
test_vector = vectorizer.transform(test_text)

# split to train and test 
train, validation, res_train, res_validation = train_test_split(bow_vector, target, test_size=.1, random_state=0)

inputs['BOW'] = [train, validation, test_vector]

In [92]:
# TF-IDF

# create the transform
tfidvectorizer = TfidfVectorizer(dtype=np.float32)
# tokenize and build vocab
tfidvectorizer.fit(text)
# encode
tfidvector = tfidvectorizer.transform(text)
test_vector = tfidvectorizer.transform(test_text)

# split to train and test
train, validation, _, _ = train_test_split(tfidvector, target, test_size=.1, random_state=0)

inputs['TF-IDF'] = [train, validation, test_vector]

##### Pretrained embeddings

In [93]:
def get_embedding(text, model):
    embedding = []

    # text to list of words
    tokenizer = WordPunctTokenizer()
    token_text = [tokenizer.tokenize(line) for line in text]
    
    for line in token_text:
        vector = []
        for word in line:
            try:
                vector.append(model.get_vector(word.lower()))
            except:
                pass
        # normalize
        embedding.append(sum(vector)/len(vector))

    return pd.DataFrame(embedding)

In [94]:
# W2V

# load word2vec model
model = api.load('word2vec-google-news-300')
# count embeddings for train and test texts
embedding = get_embedding(text, model)
test_vector = get_embedding(test_text, model)

train, validation, _, _ = train_test_split(embedding, target, test_size=.1, random_state=0)

inputs['W2V'] = [train, validation, test_vector]

In [95]:
# FastText

# load fasttext model
model = api.load('fasttext-wiki-news-subwords-300')
# count embeddings for train and test texts
embedding = get_embedding(text, model)
test_vector = get_embedding(test_text, model)

train, validation, _, _ = train_test_split(embedding, target, test_size=.1, random_state=0)

inputs['Fasttext'] = [train, validation, test_vector]

##### ML classifiers

In [111]:
# compare ML classifiers

classifiers = [
    RandomForestClassifier(),
    LogisticRegression(max_iter=10000)
]
for key, inp in inputs.items():
    for classifier in classifiers:
        classifier.fit(inp[0], res_train)
        validation_prediction = classifier.predict(inp[1])
        print('| {}, {} |  accuracy {:8.3f}'.format(type(classifier), key,
                                            accuracy_score(validation_prediction, res_validation)))

| <class 'sklearn.ensemble._forest.RandomForestClassifier'>, BOW |  accuracy    0.785
| <class 'sklearn.linear_model._logistic.LogisticRegression'>, BOW |  accuracy    0.807
| <class 'sklearn.ensemble._forest.RandomForestClassifier'>, TF-IDF |  accuracy    0.784
| <class 'sklearn.linear_model._logistic.LogisticRegression'>, TF-IDF |  accuracy    0.825
| <class 'sklearn.ensemble._forest.RandomForestClassifier'>, W2V |  accuracy    0.803
| <class 'sklearn.linear_model._logistic.LogisticRegression'>, W2V |  accuracy    0.833
| <class 'sklearn.ensemble._forest.RandomForestClassifier'>, Fasttext |  accuracy    0.820
| <class 'sklearn.linear_model._logistic.LogisticRegression'>, Fasttext |  accuracy    0.814


###### The best accuracy with ML was obtained for the LogisticRegression model when using W2V embedding

In [112]:
classifier = classifiers[1]
inp = inputs['W2V']

classifier.fit(inp[0], res_train)
test_prediction = classifier.predict(inp[2])

In [113]:
result = pd.DataFrame({'Id': test_data.index, 'Predicted': test_prediction})
result.to_csv('res.csv', index=False)

##### Neural network

In [114]:
class MyDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        data = self.data['Title'].iloc[idx] + self.data['Body'].iloc[idx] + self.data['Tags'].iloc[idx]
        if 'target' in self.data.columns:
            return self.data['target'].iloc[idx], data
        else:
            return data

In [115]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

train_iter = MyDataset(training_data)
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [116]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [117]:
device = torch.device("cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = MyDataset(training_data)
dataloader = DataLoader(train_iter, batch_size=32, collate_fn=collate_batch)

In [118]:
# FC model

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [119]:
train_iter = MyDataset(training_data)
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [120]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [121]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
test_iter = MyDataset(test_data)
train_iter = MyDataset(training_data)
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)

# split to train and test 
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/  713 batches | accuracy    0.711
-----------------------------------------------------------
| end of epoch   1 | time:  7.98s | valid accuracy    0.838 
-----------------------------------------------------------
| epoch   2 |   500/  713 batches | accuracy    0.849
-----------------------------------------------------------
| end of epoch   2 | time:  7.96s | valid accuracy    0.860 
-----------------------------------------------------------
| epoch   3 |   500/  713 batches | accuracy    0.872
-----------------------------------------------------------
| end of epoch   3 | time:  7.96s | valid accuracy    0.865 
-----------------------------------------------------------
| epoch   4 |   500/  713 batches | accuracy    0.884
-----------------------------------------------------------
| end of epoch   4 | time:  7.97s | valid accuracy    0.886 
-----------------------------------------------------------
| epoch   5 |   500/  713 batches | accuracy    0.895
------

In [122]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()
    
test_prediction = []
for text in test_dataset:
    test_prediction.append(predict(text, text_pipeline))

In [123]:
result = pd.DataFrame({'Id': test_data.index, 'Predicted': test_prediction})
result.to_csv('res1.csv', index=False)